<a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso-data-management/blob/main/tutorials/01_Check_and_update_csv_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorial #1: Check and update csv files</h1>
<h3 align="right">Written by KSO Team</h3>

# Set up and requirements

In [ ]:
# @title <font size="5">↓ ឵឵<i>Install kso_data_management and its requirements</font> { vertical-output: true }

from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone kso-data-management repo
    !git clone --quiet --recurse-submodules -b main https://github.com/ocean-data-factory-sweden/kso-data-management.git
    %pip install -q --upgrade pip
    %pip install -qr <(sed '/boto3/d;/ipywidgets/d' kso-data-management/requirements.txt)
    %pip install -qr <(sed '/boto3/d;/ipywidgets/d' kso-data-management/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    os.chdir("kso-data-management/tutorials")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False

    # Install requirements
    # %pip install -q --no-warn-script-location --upgrade pip --user
    # %pip install -qr ../requirements.txt --user
    # %pip install -qr ../kso_utils/requirements.txt --user

    !jupyter nbextension install --user --py widgetsnbextension
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension install --user --py jupyter_bbox_widget
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    !jupyter nbextension install --user --py ipysheet
    !jupyter nbextension enable --user --py ipysheet

    clear_output()
    print("Running locally... you're good to go!")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Import Python packages</font> { vertical-output: true }

# Set the directory of the libraries
import sys

try:
    if "kso_utils" not in sys.modules:
        sys.path.append("..")
        import kso_utils.kso_utils

        sys.modules["kso_utils"] = kso_utils.kso_utils
        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils


# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.t1_utils as t1
from kso_utils.project import ProjectProcessor


print("Packages loaded successfully")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose your project</font> { vertical-output: true }

project_name = t_utils.choose_project()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Initiate project's database</font> { vertical-output: true }
# Save the name of the project
project = p_utils.find_project(project_name=project_name.value)
# Initiate pp
pp = ProjectProcessor(project)

# Review Sites

In [ ]:
# @title <font size="5">↓ ឵឵<i>Map sites and metadata</font> { vertical-output: true }
pp.map_sites()

## Manually update sites metadata

In [ ]:
# @title <font size="5">↓ ឵឵<i>Select the range of sites to display</font> { vertical-output: true }
sites_df, sites_range_rows, sites_range_columns = pp.select_meta_range(meta_key="sites")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Update the contents of the cells in the spreadsheet below as needed</font> { vertical-output: true }

sites_df_filtered, sites_sheet = t1.open_csv(
    df=sites_df, df_range_rows=sites_range_rows, df_range_columns=sites_range_columns
)

sites_sheet

In [ ]:
# @title <font size="5">↓ ឵឵<i>Review the changes</font> { vertical-output: true }

# Display the changes
sites_sheet_df = pp.view_meta_changes(df_filtered=sites_df_filtered, sheet=sites_sheet)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Confirm or deny the changes</font> { vertical-output: true }

# Confirm or deny the changes
pp.update_meta(sites_sheet_df, "sites")

# Review Movies

In [ ]:
# @title <font size="5">↓ ឵឵<i>Retrieve info of movies available on the server</font> { vertical-output: true }
pp.get_movie_info()

## Preview movies

In [ ]:
# @title <font size="5">↓ ឵឵<i>Display the movie</font> { vertical-output: true }
pp.preview_media()

## Automatic check of movies metadata

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose the review method</font> { vertical-output: true }
review_method = t1.choose_movie_review()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Specify the GPU availability</font> { vertical-output: true }
gpu_available = t_utils.gpu_select()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Check and update the movies information</font> { vertical-output: true }
pp.check_movies_meta(review_method=review_method.value, gpu=gpu_available.result)

## Manually update movies metadata

In [ ]:
# @title <font size="5">↓ ឵឵<i>Select the range of movies to display</font> { vertical-output: true }
movies_df, movies_range_rows, movies_range_columns = pp.select_meta_range(
    meta_key="movies"
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Update the contents of the cells in the spreadsheet below as needed</font> { vertical-output: true }
movies_df_filtered, movies_sheet = t1.open_csv(
    df=movies_df, df_range_rows=movies_range_rows, df_range_columns=movies_range_columns
)
movies_sheet

In [ ]:
# @title <font size="5">↓ ឵឵<i>Review the changes</font> { vertical-output: true }

movies_sheet_df = pp.view_meta_changes(
    df_filtered=movies_df_filtered, sheet=movies_sheet
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Confirm or deny the changes</font> { vertical-output: true }
# Confirm or deny the changes
pp.update_meta(movies_sheet_df, "movies")

# Review species

In [ ]:
# @title <font size="5">↓ ឵឵<i>Automatically check the species dataframe</font> { vertical-output: true }
species_sheet_df = pp.check_species_meta()

## Manually update species dataframe

In [ ]:
# @title <font size="5">↓ ឵឵<i>Select the range of species to display</font> { vertical-output: true }
species_df, species_range_rows, species_range_columns = pp.select_meta_range(
    meta_key="species"
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Update the contents of the cells in the spreadsheet below as needed</font> { vertical-output: true }

species_df_filtered, species_sheet = t1.open_csv(
    df=species_df,
    df_range_rows=species_range_rows,
    df_range_columns=species_range_columns,
)
species_sheet

In [ ]:
# @title <font size="5">↓ ឵឵<i>Review the changes</font> { vertical-output: true }

# Display the changes
species_sheet_df = pp.view_meta_changes(
    df_filtered=species_df_filtered, sheet=species_sheet
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Confirm or deny the changes</font> { vertical-output: true }

# Confirm or deny the changes
pp.update_meta(sites_sheet_df, "species")

In [ ]:
# END